In [1]:
import os
from google.colab import drive

drive.mount('/content/drive')
os.chdir('drive/My Drive/Colab Notebooks/mlp/project1')
!ls

Mounted at /content/drive
cast.csv		   IMDB_1.csv		 render.html
cast_df_bk.csv		   IMDB.csv		 StopWords_EN.txt
cast_df.csv		   les-miserables.json	 the_office.csv
director_df.csv		   movie_df_bk.csv	 tmp_IMDB.csv
EpisodeInfo_df_bk.csv	   movie_df.csv		 Untitled0.ipynb
EpisodeInfo_df.csv	   project1_group.ipynb
graph_les_miserables.html  project1.ipynb


In [3]:
! pip install schrutepy

     |████████████████████████████████| 5.4 MB 4.0 MB/s 
     |████████████████████████████████| 793 kB 51.7 MB/s 
     |████████████████████████████████| 380 kB 56.1 MB/s 
     |████████████████████████████████| 54 kB 2.3 MB/s 
     |████████████████████████████████| 138 kB 56.8 MB/s 
     |████████████████████████████████| 54 kB 2.2 MB/s 
     |████████████████████████████████| 63 kB 1.6 MB/s 
     |████████████████████████████████| 3.6 MB 45.0 MB/s 
  Created wheel for schrutepy: filename=schrutepy-0.1.3-py3-none-any.whl size=3492 sha256=13708eaad324dab1080cd55af45ec11d5609a8d2d0ae28718e5e49b5ed4ff19f
  Stored in directory: /root/.cache/pip/wheels/a2/9c/66/dd742444de1036201b0cc1038a134a71b36f966af8db123444
Successfully built schrutepy
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninsta

In [2]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
from lxml import etree
import re
import pandas as pd
import time
from tqdm import tqdm
from collections import Counter

import schrutepy

ModuleNotFoundError: ignored

In [4]:
office = pd.read_csv('the_office.csv')

In [5]:
def getEpisodeInfo():
  episode_names = []
  episode_urls = []

  userReviewsNums = []
  storyLines = []
  keyWords = []
  ratings_details = []
  total_votes_details = []

  ratings_distribution = []

  US_ratings = []
  US_votenums = []
  nonUS_ratings = []
  nonUS_votenums = []

  for i in range(1,10):
    url = 'https://www.imdb.com/title/tt0386676/episodes?season=%d'%i
    response = requests.get(url)
    html = etree.HTML(response.content.decode('utf-8'))

    episode_names.extend(html.xpath('//*[@id="episodes_content"]/div[2]/div[2]/div/div[2]/strong/a/text()'))
    episode_urls.extend(html.xpath('//*[@id="episodes_content"]/div[2]/div[2]/div/div[2]/strong/a/@href'))

  for u in tqdm(episode_urls):
    response = requests.get('https://www.imdb.com' + u)
    html = etree.HTML(response.content.decode('utf-8'))

    userReviewsNum = html.xpath('//*[@id="__next"]/main/div/section[1]/div/section/div/div[1]/section[5]/div[1]/a[1]/h3/span/text()')
    if len(userReviewsNum) == 0:
      userReviewsNum = html.xpath('//*[@id="__next"]/main/div/section[1]/div/section/div/div[1]/section[6]/div[1]/a[1]/h3/span/text()')
    storyLine = html.xpath('//*[@id="__next"]/main/div/section[1]/div/section/div/div[1]/section[3]/div[2]/div[1]/div[1]/div/text()')
    if len(storyLine) == 0:
      storyLine = html.xpath('//*[@id="__next"]/main/div/section[1]/div/section/div/div[1]/section[4]/div[2]/div[1]/div[1]/div/text()')
    userReviewsNums.append(userReviewsNum[0] if len(userReviewsNum) != 0 else '')
    storyLines.append(storyLine[0] if len(storyLine) != 0 else '')

    keyword = html.xpath('//*[@id="__next"]/main/div/section[1]/div/section/div/div[1]/section[3]/div[2]/div[2]/a/span/text()')
    if len(keyword) == 0:
      keyword = html.xpath('//*[@id="__next"]/main/div/section[1]/div/section/div/div[1]/section[4]/div[2]/div[2]/a/span/text()')
    keyWords.append(keyword)

    response = requests.get('https://www.imdb.com' + u + 'ratings')
    html = etree.HTML(response.content.decode('utf-8'))

    total_votes_detail = html.xpath('//*[@id="main"]/section/div/div[3]/div/table[2]/tr/td/div[2]/a/text()')
    total_votes_detail = [int("".join(list(filter(str.isdigit, x)))) if x!= '-' else '-' for x in total_votes_detail]
    ratings_detail = html.xpath('//*[@id="main"]/section/div/div[3]/div/table[2]/tr/td/div[1]/text()')
    ratings_detail = [int("".join(list(filter(str.isdigit, x))))/10 if x!= '-' else '-' for x in ratings_detail]

    ## All all_<18 all_18-29 all_30-44 all_45+ male_All male__<18 male__18-29 male__30-44 male__45+ female_All female__<18 female__18-29 female__30-44 female__45+
    ratings_details.append(ratings_detail)
    total_votes_details.append(total_votes_detail)

    ## from 10 to 1
    ratings_dist = html.xpath('//*[@id="main"]/section/div/div[3]/div/table[1]/tr/td[3]/div/div/text()')
    ratings_distribution.append([int(["".join(list(filter(str.isdigit, x)))][0]) for x in ratings_dist])

    US_ratings.append(html.xpath('//*[@id="main"]/section/div/div[3]/div/table[3]/tr[2]/td[2]/div[1]/text()')[0])
    nonUS_ratings.append(html.xpath('//*[@id="main"]/section/div/div[3]/div/table[3]/tr[2]/td[3]/div[1]/text()')[0])

    US_votenum = html.xpath('//*[@id="main"]/section/div/div[3]/div/table[3]/tr[2]/td[2]/div[2]/a/text()')
    nonUS_votenum = html.xpath('//*[@id="main"]/section/div/div[3]/div/table[3]/tr[2]/td[3]/div[2]/a/text()')
    US_votenums.append(int("".join(list(filter(str.isdigit, US_votenum[0])))))
    nonUS_votenums.append(int("".join(list(filter(str.isdigit, nonUS_votenum[0])))))

  return pd.DataFrame({'episode_name':episode_names, 'userReviewsNum':userReviewsNums, 'storyLine':storyLines, 'keyWords':keyWords, 'ratings_details':ratings_details, 'total_votes_details': total_votes_details \
                      , 'ratings_distribution': ratings_distribution, 'US_ratings':US_ratings, 'US_votenums':US_votenums,'nonUS_ratings':nonUS_ratings, 'nonUS_votenums':nonUS_votenums, })


EpisodeInfo_df = getEpisodeInfo()

100%|██████████| 188/188 [07:43<00:00,  2.47s/it]


In [6]:
## Na in total_votes_details
fillNaVotes = []
for i in range(len(EpisodeInfo_df['ratings_details'])):
  r = EpisodeInfo_df['ratings_details'].iloc[i]
  if '-' in r:
    na_index = r.index('-')
    fillNaVotes.append(EpisodeInfo_df['total_votes_details'].iloc[i][:na_index]+['-']+EpisodeInfo_df['total_votes_details'].iloc[i][na_index:])
  else:
    fillNaVotes.append(EpisodeInfo_df['total_votes_details'].iloc[i])
EpisodeInfo_df['total_votes_details'] = fillNaVotes

In [7]:
genders = ['All', 'Male', 'Female']
ages = ['All', '<18', '18-29', '30-44', '45+']
user_portrait = []
for i in genders:
  for j in ages:
      user_portrait.append(i+'_'+j)


for i in range(len(user_portrait)):
  EpisodeInfo_df[user_portrait[i]+'_rating'] = EpisodeInfo_df['ratings_details'].apply(lambda x: x[i])
  EpisodeInfo_df[user_portrait[i]+'_totalVotesNum'] = EpisodeInfo_df['total_votes_details'].apply(lambda x: x[i])

del EpisodeInfo_df['total_votes_details']
del EpisodeInfo_df['ratings_details']

In [8]:
EpisodeInfo_df

,episode_name,userReviewsNum,storyLine,keyWords,ratings_distribution,US_ratings,US_votenums,nonUS_ratings,nonUS_votenums,All_All_rating,...,Female_All_rating,Female_All_totalVotesNum,Female_<18_rating,Female_<18_totalVotesNum,Female_18-29_rating,Female_18-29_totalVotesNum,Female_30-44_rating,Female_30-44_totalVotesNum,Female_45+_rating,Female_45+_totalVotesNum
0,Pilot,25,The regional manager of a mid-range paper supp...,"[pudding, boss, office, team]","[1170, 609, 1958, 2326, 899, 313, 129, 60, 72,...",7.6,1412,7.2,2536,7.4,...,7.3,681,8.5,4,7.3,213,7.2,295,7.6,128
1,Diversity Day,11,When Michael imitates a Chris Rock comedy rout...,"[reference to abraham lincoln, tolerance, semi...","[1818, 1625, 2313, 1016, 388, 139, 53, 30, 42,...",8.5,1456,7.9,2378,8.2,...,8.0,681,8.7,3,8.1,212,7.9,301,8.3,126
2,Health Care,8,When Michael is forced to pick a low-grade hea...,"[office, healthcare]","[887, 712, 2194, 1646, 485, 156, 60, 21, 41, 107]",8.0,1207,7.5,2084,7.7,...,7.7,550,8.3,3,7.8,167,7.5,243,7.8,104
3,The Alliance,7,Since the rumors of downsizing have occurred i...,[box],"[970, 899, 2366, 1215, 347, 103, 42, 13, 26, 107]",8.2,1169,7.8,1993,7.9,...,7.9,533,8.7,3,8.1,166,7.8,231,8.0,101
4,Basketball,8,Michael and his staff challenge the warehouse ...,"[reference to lord of the rings, game, basketb...","[1653, 1662, 2070, 801, 310, 85, 33, 14, 23, 120]",8.5,1284,8.1,2136,8.3,...,8.1,573,8.7,3,8.3,179,8.0,243,8.2,111
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
183,Stairmageddon,5,The staff uses the steps when the elevator is ...,"[episode title spoken by character, stairs, tr...","[694, 445, 1091, 590, 184, 65, 30, 17, 15, 78]",7.9,643,7.7,1043,7.9,...,8.0,294,-,-,8.0,96,7.7,110,8.6,73
184,Paper Airplane,14,The employees hold a paper airplane competitio...,"[paper airplane, eyepatch, hug, kiss]","[749, 443, 1148, 558, 183, 67, 36, 20, 16, 82]",7.9,654,7.7,1052,7.9,...,8.1,303,-,-,8.4,97,7.7,114,8.4,73
185,Livin' the Dream,14,Dwight becomes regional manager after Andy qui...,"[nbc, husband wife relationship, karate, refer...","[2395, 1335, 651, 218, 79, 30, 12, 9, 12, 113]",8.8,811,8.8,1411,9.0,...,8.9,387,9.0,1,9.1,127,8.5,152,9.1,81
186,A.A.R.M.,7,Dwight prepares for a marriage proposal and hi...,"[nbc, marriage proposal, husband wife relation...","[4442, 1132, 425, 149, 56, 31, 18, 7, 12, 183]",9.3,1069,9.3,1795,9.4,...,9.4,584,10.0,1,9.6,199,9.2,234,9.3,101


In [9]:
EpisodeInfo_df.to_csv('crawler_data/EpisodeInfo_df')

In [ ]:
## actor detiails

## info
def getInfo(url, cast_type_index = 6):
    baseurl = 'https://www.imdb.com/'
    response = requests.get(baseurl + url[1:])
    html = etree.HTML(response.content.decode('utf-8'))
    
    name = html.xpath('//*[@id="name-overview-widget-layout"]/tbody/tr[1]/td/h1/span/text()')
    if not name:
        name = html.xpath('//*[@id="overview-top"]/div[1]/div/h1/span/text()')
    name = name[0]
    born_year = html.xpath('//*[@id="name-born-info"]/time/a[2]/text()')
    born_year = born_year[0] if len(born_year)!=0 else '' 
    
    Filmography = html.xpath('//*[@id="filmography"]/div[%s]/div/b/a/text()'%cast_type_index)
    Filmography_str = ';'.join(Filmography)
    Filmography_url = html.xpath('//*[@id="filmography"]/div[%s]/div/b/a/@href'%cast_type_index)
    Filmography_url = [baseurl + x[1:] for x in Filmography_url]
    Filmography_year = html.xpath('//*[@id="filmography"]/div[%s]/div/span/text()'%cast_type_index)
    Filmography_year = [x[2:-1].replace('/I','') for x in Filmography_year]
    
    Filmography_dict = {}
    for i in range(len(Filmography)):
        Filmography_dict[Filmography[i]] = {'url': Filmography_url[i], 'year': Filmography_year[i]}
    
    # bio
    response_bio = requests.get(baseurl + url + 'bio')
    html_bio = etree.HTML(response_bio.content.decode('utf-8'))
    bio = ' '.join(html_bio.xpath('//*[@id="bio_content"]/div[2]/p[1]/text()'))  
    
    df_info = pd.DataFrame({'name': name, 'born_year': born_year, 'bio':bio, 'Filmography': Filmography_str}, index = [name])
    dict_episode = Filmography_dict
    return df_info, dict_episode

In [10]:
## actors
from lxml import etree
import requests

cast_url = 'https://www.imdb.com/title/tt0386676/fullcredits/?ref_=tt_cl_sm'
response = requests.get(cast_url)
html = etree.HTML(response.content.decode('utf-8'))

cast = []
cast_url = []
chars = []
for i in tqdm(range(2,1575)):
    table = html.xpath('//*[@id="fullcredits_content"]/table[3]')
    
    cast.append(table[0].xpath('.//tr[%d]/td[2]//text()'%i))
    chars.append(table[0].xpath('.//tr[%d]/td[4]/a[1]//text()'%i))
    cast_url.append(table[0].xpath('.//tr[%d]/td[2]//@href'%i))

cast_url = [x[0] for x in cast_url if len(x) != 0]
cast = [x[1][1:-1] for x in cast if len(x) != 0]
chars = [x[0] for x in chars if len(x) != 0]

cast_chars_df = pd.DataFrame({'cast':cast, 'chars':chars, 'cast_url':cast_url})

main_chars = list(set([y for x in list(office['main_chars'].apply(lambda x : x.split(';'))) for y in x]))

100%|██████████| 1573/1573 [00:28<00:00, 55.53it/s]


In [11]:
## directors
cast_url = 'https://www.imdb.com/title/tt0386676/fullcredits/?ref_=tt_cl_sm'
response = requests.get(cast_url)
html = etree.HTML(response.content.decode('utf-8'))
director = []
director_url = []
for i in tqdm(range(1, 56)):
    table = html.xpath('//*[@id="fullcredits_content"]/table[1]')
    director.append(table[0].xpath('.//tr[%d]/td[1]//text()'%i))
    director_url.append(table[0].xpath('.//tr[%d]/td[1]//@href'%i))
director_url = [x[0] for x in director_url if len(x) != 0]
director = [x[1][1:-1] for x in director if len(x) != 0]
director_df = pd.DataFrame({'director':director, 'director_url':director_url})

100%|██████████| 55/55 [00:00<00:00, 72.74it/s]


In [22]:
director_df_info = pd.DataFrame()
dict_episode_director = {}

for i in tqdm(range(0, len(director_df))):
    director_df_i, dict_episode_director_i = getInfo(director_df['director_url'].iloc[i])
    director_df_info = pd.concat((director_df_info, director_df_i), axis = 0)
    dict_episode_director[i] = dict_episode_director_i
director_df_info 

100%|██████████| 55/55 [02:24<00:00,  2.62s/it]


,name,born_year,bio,Filmography
Paul Feig,Paul Feig,1962,\n Paul Feig is an American film dire...,The School for Good and Evil;Welcome to Flatch...
Randall Einhorn,Randall Einhorn,1963,\n Randall Einhorn was born on Decemb...,辦公室瘋雲;The Quest;Dog Bites Man;Coachella;NASCAR...
Ken Kwapis,Ken Kwapis,1957,\n Ken Kwapis is an award-winning dir...,Sexual Life
Greg Daniels,Greg Daniels,1963,"\n Greg Daniels was born on June 13, ...",上傳天地;People of Earth;公園與遊憩;The Mindy Project;阿...
Jeffrey Blitz,Jeffrey Blitz,,\n Jeffrey Blitz is an American direc...,Review;Table 19;Rocket Science
Ken Whittingham,Ken Whittingham,,,Moesha;Malcolm & Eddie
David Rogers,David Rogers,,,上傳天地;太空部隊;The Mindy Project;辦公室瘋雲
Charles McDougall,Charles McDougall,,\n Charles McDougall is a director an...,Ana
Matt Sohn,Matt Sohn,,,Home Economics;Abbott Elementary;American Auto...
Paul Lieberstein,Paul Lieberstein,1967,\n Paul Lieberstein was born on Febru...,"Middlespace: The Rebels Attack, and Then the O..."


In [23]:
dict_set_episode_director = {}
for k,v in dict_episode_director.items():
  for k_i, v_i in v.items():
    if k_i not in dict_set_episode_director.keys():
        dict_set_episode_director[k_i] = v_i

In [24]:
director_df_info.to_csv('crawler_data/director_df.csv')

In [13]:
cast_chars_df['first_char_name'] = cast_chars_df['chars'].apply(lambda x: 1 if x.split(' ')[0]in main_chars else 0)
cast_chars_df_main = cast_chars_df[cast_chars_df['first_char_name']==1][['cast', 'chars', 'cast_url']].head(len(main_chars))

In [19]:
cast_df = pd.DataFrame()
dict_episode = {}

for i in tqdm(range(0, len(cast_chars_df_main))):
    cast_df_i, dict_episode_i = getInfo(cast_chars_df_main['cast_url'].iloc[i])
    cast_df = pd.concat((cast_df, cast_df_i), axis = 0)
    dict_episode[i] = dict_episode_i


100%|██████████| 17/17 [00:52<00:00,  3.09s/it]


In [20]:
dict_set_episode = {}
for k,v in dict_episode.items():
  for k_i, v_i in v.items():
    if k_i not in dict_set_episode.keys():
        dict_set_episode[k_i] = v_i

In [21]:
cast_df.to_csv('crawler_data/cast_df.csv')

In [25]:
## actor_movies

def getEpisodeInfo(url):
    response = requests.get(url)
    html = etree.HTML(response.content.decode('utf-8'))
    
    name = html.xpath('//*[@id="__next"]/main/div/section[1]/section/div[3]/section/section/div[1]/div[1]/h1/text()')
    name = name[0] if len(name) != 0 else '' 

    rating = html.xpath('//*[@id="__next"]/main/div/section[1]/section/div[3]/section/section/div[1]/div[2]/div/div[1]/a/div/div/div[2]/div[1]/span[1]/text()')
    rating = rating[0] if len(rating) != 0 else '' 
    
    total_votes = html.xpath('//*[@id="__next"]/main/div/section[1]/section/div[3]/section/section/div[1]/div[2]/div/div[1]/a/div/div/div[2]/div[3]/text()')
    total_votes = total_votes[0] if len(total_votes) != 0 else ''
    
    user_reviews = html.xpath('//*[@id="__next"]/main/div/section[1]/section/div[3]/section/section/div[3]/div[2]/div[2]/ul/li[1]/a/span/span[1]/text()')
    user_reviews = user_reviews[0] if len(user_reviews) != 0 else ''
    
    critic_reviews = html.xpath('//*[@id="__next"]/main/div/section[1]/section/div[3]/section/section/div[3]/div[2]/div[2]/ul/li[2]/a/span/span[1]/text()')
    critic_reviews = critic_reviews[0] if len(critic_reviews) != 0 else ''
    
    tags = ';'.join(html.xpath('//*[@id="__next"]/main/div/section[1]/section/div[3]/section/section/div[3]/div[2]/div[1]/div[1]/div/a/span/text()'))
    
    storyLine = html.xpath('//*[@id="__next"]/main/div/section[1]/div/section/div/div[1]/section[6]/div[2]/div[1]/div[1]/div/text()')
    storyLine = storyLine[0] if len(storyLine) != 0 else ''
    
    # key words
    response_kw = requests.get(url + 'keywords')
    html_kw = etree.HTML(response_kw.content.decode('utf-8'))
    keyWords = ';'.join(html_kw.xpath('//*[@id="keywords_content"]/table/tbody/tr/td/div/a/text()'))
    
    
    episodeInfo = pd.DataFrame({'name': name, 'rating':rating, 'total_votes':total_votes, 'user_reviews':user_reviews, 'critic_reviews':critic_reviews, 'tags':tags, 'storyLine':storyLine, 'keyWords':keyWords}, index = [name])
    
    return episodeInfo

In [29]:
def getQuotes_batch(urls, start_index, size = 100):
    movie_df = pd.DataFrame()
    for i in range(start_index, start_index+size):
        ur = urls[i]
        movie_df_i = getEpisodeInfo(ur)
        movie_df = pd.concat((movie_df,movie_df_i),axis = 0)
    movie_df.to_csv('crawler_data/movie_df.csv', mode = 'a', header=False)
    return movie_df

In [31]:
urls = [u['url'] for u in list(dict_set_episode.values())]
print(len(urls))
urls = urls + [u['url'] for u in list(dict_set_episode_director.values())]
print(len(urls))

135
564


In [ ]:
movie_df = pd.DataFrame()
for i in tqdm(range(len(urls))):
    ur = urls[i]
    movie_df_i = getEpisodeInfo(ur)
    movie_df = pd.concat((movie_df,movie_df_i),axis = 0)


  7%|▋         | 37/564 [01:50<24:38,  2.80s/it]

In [ ]:
movie_df.to_csv('crawler_data/movie_df.csv')